In [ ]:
# !pip install nltk
# !pip install spacy
#!python -m spacy download pt_core_news_sm

In [105]:
"""
João Assalim
https://www.kaggle.com/datasets/augustop/portuguese-tweets-for-sentiment-analysis?select=NoThemeTweets.csv
"""

'\nJoão Assalim\nhttps://www.kaggle.com/datasets/augustop/portuguese-tweets-for-sentiment-analysis?select=NoThemeTweets.csv\n'

In [97]:
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
import spacy

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

In [73]:
df = pd.read_csv('./NoThemeTweets.csv')
df = df.drop(["id", "tweet_date"], axis=1)
df = df[:50000]
df.head()

,tweet_text,sentiment,query_used
0,@Tixaa23 14 para eu ir :),Positivo,:)
1,@drexalvarez O meu like eu já dei na época :),Positivo,:)
2,Eu só queria conseguir comer alguma coisa pra ...,Positivo,:)
3,:D que lindo dia !,Positivo,:)
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Positivo,:)


In [82]:
lb = LabelEncoder()

df["sentiment"] = lb.fit_transform(df["sentiment"])
df["query_used"] = lb.fit_transform(df["query_used"])

In [75]:
df.shape

(50000, 3)

In [76]:
df.isnull().sum()

tweet_text    0
sentiment     0
query_used    0
dtype: int64

In [49]:
nlp = spacy.load("pt_core_news_sm")

In [50]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [51]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [77]:
tweets = df["tweet_text"]
tokenization = [nlp(text.lower()) for text in tweets]

In [78]:
lemmatizer = WordNetLemmatizer()

new_tweets = []

for phrase in tokenization:
  new_phrase = ""
  for token in phrase:
    if not str(token) in stop_words and not token.is_punct and "@" not in str(token) and "http" not in str(token):
      new_phrase += lemmatizer.lemmatize(str(token)) + " "
  new_tweets.append(new_phrase[:-1])

In [ ]:
new_tweets

In [79]:
df["tweet_text"] = new_tweets

In [84]:
df.head()

,tweet_text,sentiment,query_used
0,14 ir,1,1
1,like dei época,1,1
2,queria conseguir comer alguma coisa pra poder ...,1,1
3,d lindo dia,1,1
4,pq pr jeito!!é oferta ha q aproveitar :p,1,1


In [92]:
vect_uni_cv = CountVectorizer(ngram_range=(1,1), stop_words=stop_words)
text_vect_uni_cv = vect_uni_cv.fit_transform(df["tweet_text"])

X_trainUCV, X_testUCV, y_trainUCV, y_testUCV = train_test_split(text_vect_uni_cv, df["sentiment"], test_size=0.2, random_state=42)

vect_uni_idf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l2', stop_words=stop_words)
text_vect_uni_idf = vect_uni_idf.fit_transform(df["tweet_text"])

X_trainUIDF, X_testUIDF, y_trainUIDF, y_testUIDF = train_test_split(text_vect_uni_idf, df["sentiment"], test_size=0.2, random_state=42)

In [103]:
rfcUCV = RandomForestClassifier()

rfcUCV.fit(X_trainUCV, y_trainUCV)
y_predUCV = rfcUCV.predict(X_testUCV)

acUCV = accuracy_score(y_testUCV, y_predUCV)

print(f'Score Count Vectorizer Random Forest: {acUCV*100:.2f}%')

Score Count Vectorizer Random Forest: 99.99%


In [104]:
rfcidf = RandomForestClassifier()

rfcidf.fit(X_trainUIDF, y_trainUIDF)
y_predidf = rfcidf.predict(X_testUIDF)

acidf = accuracy_score(y_testUIDF, y_predidf)

print(f'Score TFIDF Random Forest: {acidf*100:.2f}%')

Score TFIDF Random Forest: 99.99%


In [99]:
dtrUVC = DecisionTreeClassifier()

dtrUVC.fit(X_trainUCV, y_trainUCV)

acUCV = dtrUVC.score(X_testUCV, y_testUCV)

print(f'Score Count Vectorizer Decision Tree Classifier: {acUCV*100:.2f}%')

Score Decision Tree Classifier: 99.86%


In [102]:
dtridf = DecisionTreeClassifier()

dtridf.fit(X_trainUIDF, y_trainUIDF)

acidf = dtridf.score(X_testUCV, y_testUCV)

print(f'Score TFIDF Decision Tree Classifier: {acidf*100:.2f}%')

Score TFIDF Decision Tree Classifier: 99.61%
